In [53]:
import tweepy
import numpy as np
import pandas as pd
import csv
# import re 
import string
# import preprocessor as p
# import swifter
import requests
from requests_oauthlib import OAuth1
import json
from tqdm.notebook import tqdm
import os
import time
from google.cloud import storage

In [54]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "even-hull-328204-0e032e418c72.json"
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAAxFUwEAAAAA9U4m4uvkk6%2FjGPrTFnZRykbnZfs%3DcsYq8RvBCSrDBAhxtRg4p6tiRRebcbQ0mEyqbXX9vxXOroWVcf',
                      wait_on_rate_limit=True)

In [55]:
CONSUMER_KEY = 'ENJ3fXY9A2AncNwIpTwzAM4Mx'
CONSUMER_SECRET = 'haYK0wRsXOiP1JU8QF2QcDtQtWroltcTcPxKYjJ95PP1vV5u5b'
OAUTH_TOKEN= '408602898-HYpc9JUNnjjTkrkEZ8MhAdSzRAjQvD3XJ2DLJJMK'
OAUTH_TOKEN_SECRET = 'J7IVJuDIrGPQe08pBieeUpn3mqvBtHUFU4iNkS3QpBQb4'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

# tweet = api.get_status('1213330173736738817')
# print(tweet.text)

In [56]:
def upload_to_gcp(out_df, monthyear):
    storage_client = storage.Client()
    bucket_name = 'bigdata_project_hksv'
    bucket = storage_client.bucket(bucket_name)
    time1 = time.strftime("%Y%m%d-%H.%M.%S")
    name = 'tweetData/'+str(monthyear)+'_'+time1+'.csv'
    blob = bucket.blob(name)
    blob.upload_from_string(out_df.to_csv(), 'text/csv')

In [57]:
def twitter_api(monthyear, tweet_id_list, out_df):
    try:
        out_df.set_index('tweet_id', inplace = True)

        tweet_list = client.get_tweets(tweet_id_list)
        ids = [str(i.id) for i in tweet_list.data]
        text = [i.text for i in tweet_list.data]

        out_df.loc[ids, 'tweet'] = text
    
        return out_df
        
    except Exception as e:
        print(e)
        if '429' in str(e):
            print(429)
        pass
            
        

In [58]:
def check_df(df2):
    if os.path.exists('logs.json'):
        with open('logs.json') as json_file:
            dict1 = json.load(json_file)
        df2 = df2[df2.tweet_id.isin((set(dict1['ids']) - set(df2.tweet_id.to_list())))]
        return df2
    else:
        return df2
    

In [59]:
def preprocessing(chunk):
    df1 = chunk[chunk.columns[0]].str.split(expand = True)
    df1.rename(columns={0: 'tweet_id', 1: 'tdate', 2: 'ttime', 3: 'tlang', 4: 'tcountry_place'}, inplace = True)
    df2 = df1[df1['tlang'] == 'en']
    
    df2 = check_df(df2)
    
    #grouping dfs by month  and year
    df2.tdate = pd.to_datetime(df2.tdate)
    df2['month_year'] = df2['tdate'].dt.to_period('M')
    df3 = df2.groupby(['month_year'])
    return df3

In [60]:
def write_to_json(appended_data):
    appended_data.reset_index(inplace = True)
    if os.path.exists('logs.json'):
        with open('logs.json') as json_file:
            dict1 = json.load(json_file)
        for i in appended_data['tweet_id'].to_list():
            dict1['ids'].append(i)
        
        f = open('logs.json', "w")
        json.dump(dict1, f)
        f.close()
    
    else:
        dict1 = {}
        dict1['ids'] = appended_data['tweet_id'].to_list()
        f = open('logs.json', "w")
        json.dump(dict1, f)
        f.close()
    

In [ ]:
chunksize = 1000000

tweets = []


for chunk in tqdm(pd.read_csv('../data/full_dataset_clean.tsv', 
                         chunksize=chunksize, 
                         iterator=True)):


    df3 = preprocessing(chunk)
    
    for monthyear in df3.groups.keys():
        print(monthyear)
        df4 = df3.get_group(monthyear)
        tweet_id_list = list(map(int, df4['tweet_id'].tolist()))
        df4.reset_index(inplace  = True, drop = True)
        appended_data = []
        for i in range(0, df4.shape[0], 100):
            tweet_ids = tweet_id_list[i: i+100]
            out_df = twitter_api(monthyear,tweet_ids,df4.iloc[i:i+100])
            appended_data.append(out_df)
        appended_data = pd.concat(appended_data)
        upload_to_gcp(appended_data, monthyear)
        write_to_json(appended_data)
            

0it [00:00, ?it/s]

## Experimentation with DASK

In [ ]:
from dask import dataframe as dd
import numpy as np
# start = time.time()
dask_df = dd.read_csv('../data/full_dataset_clean.tsv')
# end = time.time()
# print("Read csv with dask: ",(end-start),"sec")

In [ ]:
dask_df.shape

In [ ]:
df2 = dask_df[dask_df.columns[0]].str.split(n=4, expand = True)


df3 = df2.rename(columns={0: 'tweet_id', 1: 'tdate', 2: 'ttime', 3: 'tlang', 4: 'tcountry_place'})


In [ ]:
df4 = df3[df3['tlang'] == 'en']

In [ ]:
def get_tweet(id):
    try:
#         print(id)
        return api.get_status(id).text
    except:
        return np.nan

In [ ]:
df4['tweet'] = df4['tweet_id'].apply(lambda x: get_tweet(x))